In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col
import psycopg2

# PostgreSQL connection parameters
pg_host = "pgwkpdgsfwsfwvqr.postgres.database.azure.com"
pg_port = "5432"
pg_db = "fsi_db"
pg_user = "pgadmin"
pg_password = "S7rong!Passw0rd"
pg_schema = "mkl_bank"
catalog_name = "mkl_bank"
uc_schema = "default"
# Azure Storage parameters
storage_account_name = "dlstwkpdgsfwsodgm7j6jy6"
#storage_account_key = dbutils.secrets.get("<scope_name>", "<key_name>")
container_name = "bronze"

# Set up the connection to PostgreSQL
conn = psycopg2.connect(
    host=pg_host,
    port=pg_port,
    dbname=pg_db,
    user=pg_user,
    password=pg_password
)

# Get the list of tables in the schema
cur = conn.cursor()
cur.execute(f"SELECT table_name FROM information_schema.tables WHERE table_schema = '{pg_schema}'")
tables = cur.fetchall()
cur.close()

# Function to read table from PostgreSQL and write to Azure Storage
def ingest_table(table_name: str):
    jdbc_url = f"jdbc:postgresql://{pg_host}:{pg_port}/{pg_db}"
    connection_properties = {
        "user": pg_user,
        "password": pg_password,
        "driver": "org.postgresql.Driver"
    }
    
    df = spark.read.jdbc(url=jdbc_url, table=f"{pg_schema}.{table_name}", properties=connection_properties)
    
    # Write the DataFrame to Azure Storage
    #df.write.format("delta").mode("overwrite").save(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{pg_db}/{pg_schema}/{table_name}")
    # Write the DataFrame to a Unity Catalog managed table
    df.write.format("delta").mode("overwrite").saveAsTable(
        f"{catalog_name}.{uc_schema}.bronze_{table_name}"
    )

# Loop through all tables and ingest them
for table in tables:
    table_name = table[0]
    ingest_table(table_name)

# Close the PostgreSQL connection
conn.close()